<a href="https://colab.research.google.com/github/artemisIsNotBeam/llama3NoteBook/blob/main/llama_3_api_localhost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install pyngrok
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

**HF account Configuration**

this needs a config.json that looks


```
{
  "HF_TOKEN":your token here
}
```

In [5]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]
password=config_data["password"]
ngrokToken = config_data["ngrokToken"]

In [6]:
model_name = "meta-llama/Meta-Llama-3-8B"

**Quantisation Configuration**

This shrinks the data, under the max of colab. Remove this for your own computer

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Loading the Tokenizer and the LLM**

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [10]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [11]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text[len(prompt):]

In [12]:
print(get_response("what is 2+3="))

4 called? What is 2+3=4 called?
what is 2+3=4 called? What is 2+3=4 called?
what is 2+3=4 called? What is 2+3=4 called?
what is 2+3=4 called? What is 2+3=4 called?
what is 2+3=4 called? What is 2+3=4 called?
what is 2+3=4 called? What is 2+3=4 called?
what is 2+3=4 called? What is 2+


In [ ]:
from flask import Flask,request,jsonify
from pyngrok import ngrok

If running on collab, click first link, looks like (https://b08c-34-143-174-249.ngrok-free.app)

If on ur own computer, click the second link (should be local host)

In [ ]:
ngrok.set_auth_token(ngrokToken)
publicUrl = ngrok.connect(5000).public_url
app = Flask(__name__)

@app.route("/", methods=['POST','GET'])
def home():
  if request.method == 'POST':
    data = request.json
    if request.json["password"] == password:
      response = get_response(data["prompt"])
      return jsonify(response)
    else:
      return "wrong password"
  else:
    return "if you uk what the password is, send it in along with a post request"




print(f"To acess the link click {publicUrl}")
app.run()